In [18]:
import warnings
warnings.filterwarnings('ignore')

import os
from duneanalytics import DuneAnalytics
from datetime import date, datetime
from dateutil import tz
import pandas as pd
import pandas_datareader as reader
import yfinance as yf

In [19]:
%run helper.py

In [20]:
# get Dune Analytics login credentials
MY_USERNAME = os.environ.get('DUNE_USERNAME')
MY_PASSWORD = os.environ.get('DUNE_PASSWORD')
dune = DuneAnalytics(MY_USERNAME, MY_PASSWORD)

dune.login()
dune.fetch_auth_token()

# query daily prices for GLP and TriCrypto
glp_arbi_prices = dune.get_execution_result(dune.query_result_id_v3(1069389))
tricrypto_prices = dune.get_execution_result(dune.query_result_id_v3(1145739))
df_glp_prices = extract_frame_from_dune_data(glp_arbi_prices) \
    .rename({'price':'GLP'}, axis=1)
df_tri_prices = extract_frame_from_dune_data(tricrypto_prices) \
    .rename({'price':'TriCrypto'}, axis=1)
# TriCrypto price became available on 2021-06-09 and GLP on 2021-08-31. 
# let's cut TriCrypto's price data using 2021-08-31. This will ensure the 
# monthly returns to be calculated over the same months.
df_tri_prices = df_tri_prices.loc[df_glp_prices.index[0]:, :]

In [21]:
df_glp_prices.head()

,GLP
date,
2021-08-31,1.178252
2021-09-01,1.259457
2021-09-02,1.278878
2021-09-03,1.315815
2021-09-04,1.308367


In [22]:
df_tri_prices.head()

,TriCrypto
date,
2021-08-31,1642.373160
2021-09-01,1657.828150
2021-09-02,1722.377497
2021-09-03,1753.074564
2021-09-04,1750.168307


## Get Price Data from Yahoo

SP500, Reit, Tips, Bonds, Gold, Broad Commodities, BTC, and ETH 

We want to use the start date of the asset with the least amount of history as the start date of the period we want to download data for all assets. This saves time.

In [23]:
start = date(2021, 8, 31) # GLP price has the youngest history and it 
    # first became available on 2021-08-31.
today = datetime.now(tz=tz.UTC)
end = date(today.year, today.month, 1)
tickers_names = {
    '^GSPC': 'SP500',
    'VNQ': 'Real Estate',           
    'TIP': 'Inflation-Linked Bonds',   
    'BND': 'Nominal Bonds', 
    'GLD': 'Gold',
    '^SPGSCI': 'Broad Commodities',
    'BTC-USD':'BTC', 
    'ETH-USD':'ETH'
}
tickers = list(tickers_names.keys())

# downloads prices since `start` (including `start`) 
df_prices = yf.download(tickers, start, end)['Adj Close'] \
    .rename(tickers_names, axis=1)
df_prices.columns.name = None

[*********************100%***********************]  8 of 8 completed


In [24]:
df_prices.head(2)

,Nominal Bonds,BTC,ETH,Gold,Inflation-Linked Bonds,Real Estate,SP500,Broad Commodities
Date,,,,,,,,
2021-08-31,83.558563,47166.687500,3433.732666,169.690002,119.410301,103.063004,4522.680176,527.369995
2021-09-01,83.573067,48847.027344,3834.828125,169.699997,119.462265,104.714600,4524.089844,526.090027


In [25]:
df_prices.tail(2)

,Nominal Bonds,BTC,ETH,Gold,Inflation-Linked Bonds,Real Estate,SP500,Broad Commodities
Date,,,,,,,,
2022-12-30,71.839996,16602.585938,1199.232788,169.639999,106.440002,82.480003,3839.5,610.070007
2022-12-31,NaN,16547.496094,1196.771240,NaN,NaN,NaN,NaN,NaN


In [26]:
# drop the last row since end date is the first day of the current month, keeping it will result a fake current month return
df_prices = df_prices.iloc[:-1]

In [27]:
df_prices.tail(2)

,Nominal Bonds,BTC,ETH,Gold,Inflation-Linked Bonds,Real Estate,SP500,Broad Commodities
Date,,,,,,,,
2022-12-29,72.139999,16642.341797,1201.595337,168.850006,106.760002,83.080002,3849.280029,602.840027
2022-12-30,71.839996,16602.585938,1199.232788,169.639999,106.440002,82.480003,3839.500000,610.070007


In [28]:
# download risk free rates, which are already multiplied by 100, so we divide by 100
# behaves correctly starting on `start` not the day before
rfs = reader.DataReader('F-F_Research_Data_Factors', 'famafrench', start, end)[0].RF / 100 
rfs.head()

Date
2021-08    0.0000
2021-09    0.0000
2021-10    0.0000
2021-11    0.0000
2021-12    0.0001
Freq: M, Name: RF, dtype: float64

## Calculate Monthly Excess Returns

In [29]:
monthly_rets = df_prices.resample('M').last().pct_change()
monthly_rets_glp = df_glp_prices.resample('M').last().pct_change()
monthly_rets_tri = df_tri_prices.resample('M').last().pct_change()
monthly_rets = monthly_rets.join(monthly_rets_glp).join(monthly_rets_tri)

In [30]:
monthly_rets.head(2)

,Nominal Bonds,BTC,ETH,Gold,Inflation-Linked Bonds,Real Estate,SP500,Broad Commodities,GLP,TriCrypto
Date,,,,,,,,,,
2021-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-30,-0.010136,-0.071572,-0.125826,-0.032235,-0.007801,-0.056815,-0.047569,0.057531,-0.036346,-0.067016


In [31]:
monthly_rets_glp.head(2)

,GLP
date,
2021-08-31,NaN
2021-09-30,-0.036346


In [32]:
monthly_rets_tri.head(2)

,TriCrypto
date,
2021-08-31,NaN
2021-09-30,-0.067016


In [33]:
monthly_rets = monthly_rets.to_period('M') # because the rfs have monthly period, otherwise can't join
monthly_rets = monthly_rets.join(rfs)

In [34]:
# calculate monthly excess returns
for col in monthly_rets.columns.drop('RF'):
    newcol = col + ' - ' + 'RF'
    monthly_rets[newcol] = monthly_rets[col] - monthly_rets['RF']
# ensure all assets have the same months for fair comparison.  
excess_monthly_rets = monthly_rets.dropna().loc[:, monthly_rets.columns.str.endswith('- RF')]
# remove ' - RF' from the column names for better display
excess_monthly_rets.columns = excess_monthly_rets.columns.str.replace(' - RF', '')
excess_monthly_rets.head()

,Nominal Bonds,BTC,ETH,Gold,Inflation-Linked Bonds,Real Estate,SP500,Broad Commodities,GLP,TriCrypto
Date,,,,,,,,,,
2021-09,-0.010136,-0.071572,-0.125826,-0.032235,-0.007801,-0.056815,-0.047569,0.057531,-0.036346,-0.067016
2021-10,0.000703,0.400267,0.428559,0.014797,0.011096,0.071330,0.069144,0.054688,0.146687,0.261740
2021-11,0.002041,-0.070346,0.080084,-0.006901,0.008650,-0.021093,-0.008334,-0.111814,0.017250,0.006004
2021-12,-0.003204,-0.187784,-0.204969,0.032891,0.003929,0.096956,0.043513,0.074052,-0.065666,-0.128538
2022-01,-0.020649,-0.168947,-0.270012,-0.016788,-0.020588,-0.084217,-0.052585,0.111551,-0.213344,-0.185582


In [35]:
excess_monthly_rets.tail()

,Nominal Bonds,BTC,ETH,Gold,Inflation-Linked Bonds,Real Estate,SP500,Broad Commodities,GLP,TriCrypto
Date,,,,,,,,,,
2022-07,0.023018,0.178741,0.574689,-0.026682,0.042299,0.085360,0.090316,-0.024192,0.259140,0.278097
2022-08,-0.029875,-0.142756,-0.077922,-0.031333,-0.028942,-0.062025,-0.044340,-0.040463,-0.049757,-0.082236
2022-09,-0.043730,-0.032722,-0.147172,-0.030782,-0.069223,-0.130613,-0.095296,-0.089251,-0.037517,-0.061867
2022-10,-0.013877,0.052455,0.181992,-0.020144,0.011904,0.032751,0.077563,0.045548,0.072572,0.075850
2022-11,0.033759,-0.165236,-0.179045,0.082019,0.015429,0.058561,0.050853,-0.027600,-0.111699,-0.130610


In [36]:
print('Data period: ', excess_monthly_rets.index.min().strftime('%Y-%m'), 
      '~', excess_monthly_rets.index.max().strftime('%Y-%m'))
print("Number of months:", len(excess_monthly_rets))

Data period:  2021-09 ~ 2022-11
Number of months: 15


## Output Tables

Calculate Beta, Sharpe Ratio, and Excess Return (Ann) using Excess Monthly Returns
   - Treat SP500 as benchmark
   - GLP and TriCrypto Yields are excluded

In [37]:
market = 'SP500'
tokens = excess_monthly_rets.columns

betas = []
pvals = []
r2s = []
for token in tokens:
    res = calc_beta(excess_monthly_rets, token, market)
    betas.append(res['beta'])
    pvals.append(res['p-val'])
    r2s.append(res['R2'])

df_betas = pd.Series(betas, index=tokens).sort_values().to_frame().rename({0:'Beta'}, axis=1)
df_pvals = pd.Series(pvals, index=tokens).sort_values().to_frame().rename({0:'p-Value'}, axis=1)
df_r2s = pd.Series(r2s, index=tokens).sort_values().to_frame().rename({0:'R2'}, axis=1)
df_betas = df_betas.join(df_pvals).join(df_r2s)    

In [38]:
sharpe_ratios = excess_monthly_rets.mean() / excess_monthly_rets.std()
df_sharpes = sharpe_ratios.sort_values(ascending=False).to_frame().rename({0:'Sharpe Ratio'}, axis=1)

tot_ret = (1+excess_monthly_rets).prod()-1
dur_years = len(excess_monthly_rets) / 12
ann_excess_rets = annualize_tot_ret(tot_ret, dur_years) * 100
df_ann_excess_rets = ann_excess_rets.sort_values(ascending=False).to_frame().rename({0:'Excess Return (Ann)'}, axis=1)

In [39]:
df_sharpes.style.format(precision=3)

,Sharpe Ratio
Broad Commodities,0.176
Gold,-0.058
SP500,-0.090
ETH,-0.106
GLP,-0.127
Real Estate,-0.136
TriCrypto,-0.188
BTC,-0.262
Inflation-Linked Bonds,-0.271
Nominal Bonds,-0.462


In [40]:
df_ann_excess_rets.style.format({'Excess Return (Ann)': '{:,.1f}%'.format})

,Excess Return (Ann)
Broad Commodities,13.0%
Gold,-3.2%
Inflation-Linked Bonds,-8.6%
SP500,-8.7%
Nominal Bonds,-11.6%
Real Estate,-13.6%
GLP,-24.0%
TriCrypto,-41.2%
ETH,-54.6%
BTC,-55.9%


In [41]:
df_betas.style.format(precision=3)

,Beta,p-Value,R2
Gold,0.196,0.207,0.119
Nominal Bonds,0.234,0.005,0.469
Broad Commodities,0.272,0.391,0.057
Inflation-Linked Bonds,0.334,0.000,0.644
SP500,1.000,0.000,1.000
Real Estate,1.020,0.000,0.821
GLP,1.234,0.011,0.403
TriCrypto,1.666,0.009,0.418
BTC,1.683,0.028,0.321
ETH,2.967,0.005,0.463
